# 13. 영화 추천 엔진 만들기

## 협업 필터링을 이용한 영화 추천 엔진

### 데이터 준비

- MovieLens 100k Dataset

- 1682개의 영화에 대한 948명의 사용자 평가

- http://files.grouplens.org/datasets/movielens/ㅡml-100k.zip or https://grouplens.org/datasets/movielens/100k/

In [29]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("source/u.data", sep="\t", header=None)
df.columns = ["user_id", "item_id", "rating", "timestamp"]
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [10]:
df.shape

(100000, 4)

### 데이터 탐색

In [21]:
# Groupby로 rating별 user_id를 count
df.groupby(["rating"])[["user_id"]].count()

,user_id
rating,
1,6110
2,11370
3,27145
4,34174
5,21201


In [22]:
# Groupby로 item_id별 user_id를 count
# head로 상위 5개만 출력
df.groupby(["item_id"])[["user_id"]].count().head()

,user_id
item_id,
1,452
2,131
3,90
4,209
5,86


### 평가 행렬(rating)의 생성

In [24]:
n_users = df.user_id.unique().shape[0]
n_users

943

In [28]:
n_items = df.item_id.unique().shape[0]
n_items

1682

In [32]:
ratings = np.zeros((n_users, n_items))
ratings.shape

(943, 1682)

- 1 ~ 5의 평점: sparse matrix 역할 수행

- 행과 열의 정보를 가지고 평점 정보 넣기

In [42]:
iters = df.itertuples()
list(iters)[:5]

[Pandas(Index=0, user_id=196, item_id=242, rating=3, timestamp=881250949),
 Pandas(Index=1, user_id=186, item_id=302, rating=3, timestamp=891717742),
 Pandas(Index=2, user_id=22, item_id=377, rating=1, timestamp=878887116),
 Pandas(Index=3, user_id=244, item_id=51, rating=2, timestamp=880606923),
 Pandas(Index=4, user_id=166, item_id=346, rating=1, timestamp=886397596)]

In [36]:
for row in df.itertuples():
    # row[1]-1: 사용자 정보, row[2]-1: 영화 정보, row[3]: 평점
    ratings[row[1]-1, row[2]-1] = row[3]

type(ratings)

numpy.ndarray

In [37]:
ratings.shape

(943, 1682)

In [38]:
ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [44]:
# 첫번째 사용자의 첫번째 영화에 대한 평점 정보
ratings[0][0]

5.0

### 훈련 데이터와 테스트 데이터 분리하기

In [47]:
from sklearn.model_selection import train_test_split

ratings_train, ratings_test = train_test_split(ratings, test_size=0.33, random_state=42)
ratings_train.shape, ratings_test.shape

((631, 1682), (312, 1682))

<br/>

### Reference

- https://www.youtube.com/watch?v=mJ8pEjN3FKg